In [0]:
# Read all CSVs into Spark DataFrames
df_customers = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_customers_dataset.csv")
df_geolocation = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_geolocation_dataset.csv")
df_order_items = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_order_items_dataset.csv")
df_order_payments = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_order_payments_dataset.csv")
df_order_reviews = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_order_reviews_dataset.csv")
df_orders = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_orders_dataset.csv")
df_products = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_products_dataset.csv")
df_sellers = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_sellers_dataset.csv")
df_product_category_name_translation = spark.read.option("header", True).csv("dbfs:/Volumes/commerce_spark_workspace/default/ecommerce_raw/product_category_name_translation.csv")


In [0]:
df_orders = df_orders.select("order_id", "customer_id", "order_status", "order_purchase_timestamp")
df_order_items = df_order_items.select("order_id", "order_item_id", "product_id", "seller_id", "price")


In [0]:
df_customers = spark.read.option("header", True).csv("file:/dbfs/Volumes/commerce_spark_workspace/default/ecommerce_raw/olist_customers_dataset.csv")


In [0]:
# Join customers → orders → order_items → products → sellers → reviews
df_all = (df_customers
    .join(df_orders, "customer_id", "inner")
    .join(df_order_items, "order_id", "inner")
    .join(df_products, "product_id", "inner")
    .join(df_sellers, "seller_id", "inner")
    .join(df_order_reviews, "order_id", "left")  # left join for reviews (optional)
)

df_all.show(10, truncate=False)


+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+------------------------+----------------+--------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+------+-------------+----------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+--------------+------------+--------------------------------+------------+--------------------+---------------------------------------------------------------------------+--------------------+-----------------------+
|order_id                        |seller_id                       |product_id                      |customer_id                     |customer_unique_id              |customer_z

# Optimized Joins For Data Integration

In [0]:
from pyspark.sql.functions import count

# Join customers with orders
df_orders_per_customer = df_customers.join(df_orders, "customer_id", "inner") \
    .groupBy("customer_id") \
    .agg(count("order_id").alias("total_orders")) \
    .orderBy("total_orders", ascending=False)

df_orders_per_customer.show(10, truncate=False)


+--------------------------------+------------+
|customer_id                     |total_orders|
+--------------------------------+------------+
|0bd683b7ceca26b5bba4e327682275c5|1           |
|ba76714c4894372325ea2d044823344a|1           |
|f7398fc942c8fa80e5419ae52e49f7fb|1           |
|66e8039d5fddd75067de6b08e4bb22e7|1           |
|04fc2ecbb192c71163629f423d57a13d|1           |
|d0b0b2dd8bdaf36ebea19c232b4e986b|1           |
|69009aaf4e400602c05b1da3989ca40b|1           |
|b1e99a86b163f1f25e7e0fa3360ad93d|1           |
|f3457b8fdac18622de498551383ae1cc|1           |
|c2928a50aecf1bc4776082b13225e4da|1           |
+--------------------------------+------------+
only showing top 10 rows


In [0]:
from pyspark.sql.functions import avg

# Join sellers with order items, orders, and reviews
df_avg_review_per_seller = df_sellers \
    .join(df_order_items, "seller_id", "inner") \
    .join(df_orders, "order_id", "inner") \
    .join(df_order_reviews, "order_id", "inner") \
    .groupBy("seller_id") \
    .agg(avg("review_score").alias("avg_review_score")) \
    .orderBy("avg_review_score", ascending=False)

df_avg_review_per_seller.show(10, truncate=False)


+--------------------------------+----------------+
|seller_id                       |avg_review_score|
+--------------------------------+----------------+
|e1c76f339ebd5460999f25a2aa8c92c5|5.0             |
|8c9348f33ae3dada25c99c99ade2af78|5.0             |
|05730013efda596306417c3b09302475|5.0             |
|702835e4b785b67a084280efca355756|5.0             |
|4a82c4af97ffc0fb2dc26bfdc03b1842|5.0             |
|1e47defeeadeca0e9a18fa5a9311e735|5.0             |
|fd312b6bf05efac6c3772d5b52205d8a|5.0             |
|44717f64ec2a457979cf83c429077666|5.0             |
|a64e44665225d19dfc0277eeeaaccc57|5.0             |
|2fa13c8bd5705d279f7ed5cc9ec61c68|5.0             |
+--------------------------------+----------------+
only showing top 10 rows


In [0]:
from pyspark.sql.functions import count

df_top_products = df_order_items \
    .groupBy("product_id") \
    .agg(count("*").alias("total_sold")) \
    .orderBy("total_sold", ascending=False) \
    .limit(10)

df_top_products.show()


+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|       527|
|99a4788cb24856965...|       488|
|422879e10f4668299...|       484|
|389d119b48cf3043d...|       392|
|368c6c730842d7801...|       388|
|53759a2ecddad2bb8...|       373|
|d1c427060a0f73f6b...|       343|
|53b36df67ebb7c415...|       323|
|154e7e31ebfa09220...|       281|
|3dd2a17168ec895c7...|       274|
+--------------------+----------+



In [0]:
from pyspark.sql.functions import col, sum as _sum

df_top_customers = df_order_items \
    .join(df_orders, "order_id", "inner") \
    .join(df_customers, "customer_id", "inner") \
    .withColumn("total_price", col("price").cast("float") + col("freight_value").cast("float")) \
    .groupBy("customer_id") \
    .agg(_sum("total_price").alias("total_spent")) \
    .orderBy(col("total_spent").desc()) \
    .limit(10)

df_top_customers.show()


+--------------------+-----------------+
|         customer_id|      total_spent|
+--------------------+-----------------+
|1617b1357756262bf...|  13664.080078125|
|ec5b2ba62e5743423...|  7274.8798828125|
|c6e2731c5b391845f...| 6929.31005859375|
|f48d464a0baaea338...|  6922.2099609375|
|3fd6777bbce08a352...|    6726.66015625|
|05455dfa7cd02f13d...|6081.539794921875|
|df55c14d1476a9a34...|    4950.33984375|
|e0a2412720e9ea4f2...| 4809.43994140625|
|24bbf5fd2f2e1b359...|    4764.33984375|
|3d979689f636322c6...| 4681.77978515625|
+--------------------+-----------------+



## Window Functions And Ranking

In [0]:
# Top selling products per seller

from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum as _sum, rank

# Aggregate total sold per product per seller
df_seller_product_sales = df_order_items \
    .groupBy("seller_id", "product_id") \
    .agg(_sum("order_item_id").alias("total_sold"))

# Define window partitioned by seller
window_spec = Window.partitionBy("seller_id").orderBy(col("total_sold").desc())

# Rank products within each seller
df_ranked_products = df_seller_product_sales \
    .withColumn("rank", rank().over(window_spec))

df_ranked_products.show()


+--------------------+--------------------+----------+----+
|           seller_id|          product_id|total_sold|rank|
+--------------------+--------------------+----------+----+
|0015a82c2db000af6...|a2ff5a97bf95719e3...|       3.0|   1|
|001cca7ae9ae17fb1...|08574b074924071f4...|     131.0|   1|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|      67.0|   2|
|001cca7ae9ae17fb1...|98a8c2fa16d7239c6...|      33.0|   3|
|001cca7ae9ae17fb1...|547b95702aec86f05...|      23.0|   4|
|001cca7ae9ae17fb1...|0da9ffd92214425d8...|      18.0|   5|
|001cca7ae9ae17fb1...|86b22a03cb72239dd...|      12.0|   6|
|001cca7ae9ae17fb1...|21fecd254a3103704...|       3.0|   7|
|001cca7ae9ae17fb1...|4d7fee7877228c149...|       2.0|   8|
|001cca7ae9ae17fb1...|4f3b83b83f7fb280f...|       1.0|   9|
|001cca7ae9ae17fb1...|6d15a14a5c04e3ef3...|       1.0|   9|
|001cca7ae9ae17fb1...|d32f22e03fb01595a...|       1.0|   9|
|001e6ad469a905060...|093cd981b714bcdff...|       1.0|   1|
|002100f778ceb8431...|158102fe543dbaeb8.

## Advanced Aggregation and Enrichment

In [0]:
#calculate total revenue and average order value (AOV) per customer
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, sum as _sum, countDistinct, round

# Cast string columns to Double
df_order_items_casted = df_order_items \
    .withColumn("price", col("price").cast(DoubleType())) \
    .withColumn("freight_value", col("freight_value").cast(DoubleType()))

# Join with orders table to get customer_id
df_customer_orders = df_order_items_casted \
    .join(df_orders, "order_id", "inner")

# Calculate revenue and aggregation
df_customer_revenue = df_customer_orders \
    .withColumn("order_revenue", col("price") + col("freight_value")) \
    .groupBy("customer_id") \
    .agg(
        _sum("order_revenue").alias("total_revenue"),
        countDistinct("order_id").alias("total_orders")
    )

# Calculate AOV
df_customer_revenue = df_customer_revenue \
    .withColumn("AOV", round(col("total_revenue") / col("total_orders"), 2))

df_customer_revenue.show(truncate=False)


+--------------------------------+------------------+------------+------+
|customer_id                     |total_revenue     |total_orders|AOV   |
+--------------------------------+------------------+------------+------+
|f3457b8fdac18622de498551383ae1cc|166.16            |1           |166.16|
|2306cc5d9784d518dce0f1d7649b1d61|276.35            |1           |276.35|
|809c3dc9eabbbe4e6b03198d239b6e2d|205.87            |1           |205.87|
|f95a88c152cc3320d2e6ab9efb9cc90c|424.03            |1           |424.03|
|05860a17b17441a53e85b2c0e2daf737|156.16            |1           |156.16|
|c55751576fef472fbe91aa1fe165a581|69.33             |1           |69.33 |
|ee039bb5e02a2f5eab3da733e88dbec3|140.32            |1           |140.32|
|a2141dae4f56fd98148f8d8956f2c10a|70.03             |1           |70.03 |
|fdd0876705d2f479feeb936cd9cad782|721.74            |1           |721.74|
|43d9a8df81f815755d9a258f2ee3135f|144.54            |1           |144.54|
|d4d2ea890454df93e3ce1c29627135b2|72.3

In [0]:
from pyspark.sql.functions import sum, avg, countDistinct, col

# Convert columns to appropriate numeric types since CSV loads as strings by default
df_order_items = df_order_items.withColumn("price", col("price").cast("double")) \
                               .withColumn("freight_value", col("freight_value").cast("double")) \
                               .withColumn("order_id", col("order_id").cast("string"))

df_order_reviews = df_order_reviews.withColumn("review_score", col("review_score").cast("double")) \
                                   .withColumn("order_id", col("order_id").cast("string"))

df_orders = df_orders.withColumn("order_id", col("order_id").cast("string"))


In [0]:
# Filter out rows where numeric columns are null (optional but recommended)
df_order_items_filtered = df_order_items.filter(col("price").isNotNull() & col("freight_value").isNotNull())
df_order_reviews_filtered = df_order_reviews.filter(col("review_score").isNotNull())


In [0]:
# Calculate total revenue from order_items
total_revenue = df_order_items.select(
    (sum(col("price") + col("freight_value"))).alias("total_revenue")
).collect()[0]["total_revenue"]


In [0]:
# Calculate total orders from orders dataframe
total_orders = df_orders.select(
    countDistinct(col("order_id")).alias("total_orders")
).collect()[0]["total_orders"]

In [0]:
print(f"Total Revenue: {total_revenue}")

print(f"Total Orders: {total_orders}")

Total Revenue: 15843553.240000086
Total Orders: 99441


In [0]:
from pyspark.sql.functions import col, sum, avg, countDistinct, expr

# 1. Cast numeric columns properly in order_items
df_order_items = df_order_items.withColumn("product_id", col("product_id").cast("string")) \
                               .withColumn("order_id", col("order_id").cast("string")) \
                               .withColumn("price", expr("try_cast(price as double)")) \
                               .withColumn("freight_value", expr("try_cast(freight_value as double)"))

# 2. Cast numeric columns properly in order_reviews
df_order_reviews = df_order_reviews.withColumn("order_id", col("order_id").cast("string")) \
                                   .withColumn("review_score", expr("try_cast(review_score as double)"))

# 3. Join order_reviews with order_items on order_id to get product_id for each review
df_reviews_with_product = df_order_reviews.join(
    df_order_items.select("order_id", "product_id").distinct(),
    on="order_id",
    how="left"
)

# Check schema and columns after join - just to confirm
df_reviews_with_product.printSchema()
df_reviews_with_product.show(5)

# 4. Calculate average review score per product
product_reviews = df_reviews_with_product.groupBy("product_id").agg(
    avg("review_score").alias("average_review_score"),
    countDistinct("order_id").alias("review_count")
)

# 5. Calculate product sales metrics
product_sales = df_order_items.groupBy("product_id").agg(
    sum(expr("1")).alias("total_quantity_sold"),  # Assuming each row = 1 quantity
    sum(col("price") + col("freight_value")).alias("total_revenue"),
    countDistinct("order_id").alias("unique_orders")
)

# 6. Join sales and reviews to get overall product popularity
product_popularity = product_sales.join(product_reviews, "product_id", "left")

display(product_popularity)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5926941429959711>, line 21
     14 df_reviews_with_product = df_order_reviews.join(
     15     df_order_items.select("order_id", "product_id").distinct(),
     16     on="order_id",
     17     how="left"
     18 )
     20 # Check schema and columns after join - just to confirm
---> 21 df_reviews_with_product.printSchema()
     22 df_reviews_with_product.show(5)
     24 # 4. Calculate average review score per product

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2027, in DataFrame.printSchema(self, level)
   2025     print(self.schema.treeString(level))
   2026 else:
-> 2027     print(self.schema.treeString())

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2000, in DataFrame.schema(self)
   1998 if self._cached_schema is None:
   1999     que

In [0]:
df_order_items.printSchema()
df_order_items.show(5)


root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- quantity: integer (nullable = false)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+----------------+--------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|product_quantity|quantity|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+----------------+--------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|               1|       1|
|00018f77f2f0320c5...|

In [0]:
 # montly revenue and order count trend
from pyspark.sql.functions import col, sum, countDistinct, to_date, date_format, expr

# Cast needed columns
df_orders = df_orders.withColumn("order_id", col("order_id").cast("string")) \
                     .withColumn("order_purchase_timestamp", col("order_purchase_timestamp").cast("timestamp"))

df_order_items = df_order_items.withColumn("order_id", col("order_id").cast("string")) \
                               .withColumn("price", expr("try_cast(price as double)")) \
                               .withColumn("freight_value", expr("try_cast(freight_value as double)"))

# Join orders and order items on order_id
df_order_joined = df_orders.join(df_order_items, on="order_id", how="inner")

# Extract month and year from order_purchase_timestamp as YYYY-MM
df_order_joined = df_order_joined.withColumn("order_month", date_format(col("order_purchase_timestamp"), "yyyy-MM"))

# Aggregate monthly revenue and order count
monthly_trend = df_order_joined.groupBy("order_month").agg(
    sum(col("price") + col("freight_value")).alias("monthly_revenue"),
    countDistinct("order_id").alias("monthly_order_count")
).orderBy("order_month")

monthly_trend.show(truncate=False)


+-----------+------------------+-------------------+
|order_month|monthly_revenue   |monthly_order_count|
+-----------+------------------+-------------------+
|2016-09    |354.75            |3                  |
|2016-10    |56808.84          |308                |
|2016-12    |19.62             |1                  |
|2017-01    |137188.49000000005|789                |
|2017-02    |286280.6199999999 |1733               |
|2017-03    |432048.58999999973|2641               |
|2017-04    |412422.23999999976|2391               |
|2017-05    |586190.9499999995 |3660               |
|2017-06    |502963.03999999975|3217               |
|2017-07    |584971.6200000002 |3969               |
|2017-08    |668204.5999999999 |4293               |
|2017-09    |720398.9100000006 |4243               |
|2017-10    |769312.3700000001 |4568               |
|2017-11    |1179143.7700000012|7451               |
|2017-12    |863547.23         |5624               |
|2018-01    |1107301.8899999992|7220          

In [0]:
# customer retention Analysis (First and Lat Order)
from pyspark.sql.functions import col, min, max, datediff
from pyspark.sql.types import TimestampType

# Cast needed columns
df_orders = df_orders.withColumn("customer_id", col("customer_id").cast("string")) \
                     .withColumn("order_purchase_timestamp", col("order_purchase_timestamp").cast(TimestampType()))

# Aggregate to find first and last order date per customer
customer_retention = df_orders.groupBy("customer_id").agg(
    min("order_purchase_timestamp").alias("first_order_date"),
    max("order_purchase_timestamp").alias("last_order_date")
)

# Calculate retention period in days (last - first)
customer_retention = customer_retention.withColumn(
    "retention_days",
    datediff(col("last_order_date"), col("first_order_date"))
)

# Show results
customer_retention.show(truncate=False)


+--------------------------------+-------------------+-------------------+--------------+
|customer_id                     |first_order_date   |last_order_date    |retention_days|
+--------------------------------+-------------------+-------------------+--------------+
|f7398fc942c8fa80e5419ae52e49f7fb|2018-04-15 19:42:06|2018-04-15 19:42:06|0             |
|d0b0b2dd8bdaf36ebea19c232b4e986b|2018-06-08 16:47:20|2018-06-08 16:47:20|0             |
|66e8039d5fddd75067de6b08e4bb22e7|2017-12-04 17:01:15|2017-12-04 17:01:15|0             |
|c2928a50aecf1bc4776082b13225e4da|2018-01-15 18:00:35|2018-01-15 18:00:35|0             |
|ba76714c4894372325ea2d044823344a|2017-05-15 08:58:47|2017-05-15 08:58:47|0             |
|04fc2ecbb192c71163629f423d57a13d|2017-11-02 17:35:55|2017-11-02 17:35:55|0             |
|f3457b8fdac18622de498551383ae1cc|2017-09-13 18:49:50|2017-09-13 18:49:50|0             |
|0bd683b7ceca26b5bba4e327682275c5|2017-10-28 20:16:56|2017-10-28 20:16:56|0             |
|b1e99a86b

In [0]:
# Order ststus Flags
from pyspark.sql.functions import when, col, lit

# Example with df_orders, assuming these columns exist and are timestamps or strings

df_orders_flagged = df_orders \
    .withColumn("Approved", when(col("order_approved_at").isNotNull(), lit(1)).otherwise(lit(0))) \
    .withColumn("Shipped", when(col("order_delivered_carrier_date").isNotNull(), lit(1)).otherwise(lit(0))) \
    .withColumn("Delivered", when(col("order_delivered_customer_date").isNotNull(), lit(1)).otherwise(lit(0))) \
    .withColumn("Canceled", when(col("order_status") == "canceled", lit(1)).otherwise(lit(0)))

df_orders_flagged.select("order_id", "Approved", "Shipped", "Delivered", "Canceled").show(10)


+--------------------+--------+-------+---------+--------+
|            order_id|Approved|Shipped|Delivered|Canceled|
+--------------------+--------+-------+---------+--------+
|e481f51cbdc54678b...|       1|      1|        1|       0|
|53cdb2fc8bc7dce0b...|       1|      1|        1|       0|
|47770eb9100c2d0c4...|       1|      1|        1|       0|
|949d5b44dbf5de918...|       1|      1|        1|       0|
|ad21c59c0840e6cb8...|       1|      1|        1|       0|
|a4591c265e18cb1dc...|       1|      1|        1|       0|
|136cce7faa42fdb2c...|       1|      0|        0|       0|
|6514b8ad8028c9f2c...|       1|      1|        1|       0|
|76c6e866289321a7c...|       1|      1|        1|       0|
|e69bfb5eb88e0ed6a...|       1|      1|        1|       0|
+--------------------+--------+-------+---------+--------+
only showing top 10 rows


In [0]:
#Hourly order distribution
from pyspark.sql.functions import hour, col, count

# Cast timestamp column if needed
df_orders = df_orders.withColumn("order_purchase_timestamp", col("order_purchase_timestamp").cast("timestamp"))

# Extract hour from order_purchase_timestamp
df_orders_with_hour = df_orders.withColumn("order_hour", hour(col("order_purchase_timestamp")))

# Group by hour and count orders
hourly_order_dist = df_orders_with_hour.groupBy("order_hour").agg(
    count("order_id").alias("orders_count")
).orderBy("order_hour")

hourly_order_dist.show(20)


+----------+------------+
|order_hour|orders_count|
+----------+------------+
|         0|        2394|
|         1|        1170|
|         2|         510|
|         3|         272|
|         4|         206|
|         5|         188|
|         6|         502|
|         7|        1231|
|         8|        2967|
|         9|        4785|
|        10|        6177|
|        11|        6578|
|        12|        5995|
|        13|        6518|
|        14|        6569|
|        15|        6454|
|        16|        6675|
|        17|        6150|
|        18|        5769|
|        19|        5982|
+----------+------------+
only showing top 20 rows


In [0]:
# Weekday VS Weekend Orders
from pyspark.sql.functions import dayofweek, when, col, count

# Cast order_purchase_timestamp as timestamp if not already
df_orders = df_orders.withColumn("order_purchase_timestamp", col("order_purchase_timestamp").cast("timestamp"))
# Extract day of week (1=Sunday, 2=Monday, ..., 7=Saturday)
df_orders = df_orders.withColumn("day_of_week", dayofweek(col("order_purchase_timestamp")))
# Create a new column 'order_day_type' to flag Weekday vs Weekend
df_orders = df_orders.withColumn(
    "order_day_type",
    when(col("day_of_week").isin([1,7]), "Weekend").otherwise("Weekday")
)
# Group by day type and count orders
weekday_vs_weekend = df_orders.groupBy("order_day_type").agg(
    count("order_id").alias("orders_count")
)
weekday_vs_weekend.show()


+--------------+------------+
|order_day_type|orders_count|
+--------------+------------+
|       Weekday|       76594|
|       Weekend|       22847|
+--------------+------------+

